In [ ]:
# -*- coding:       utf-8 -*-
# author:           qingyang
# create_time:      2023/4/12 10:48
# 运行环境           Python3.9.6/windows/jupyter notebook
# github            https://github.com/qingyang-qingyang/weibo_spider_2023-

In [1]:
import requests

from lxml import etree
from collections import OrderedDict

from urllib.parse import quote
from urllib.parse import urlencode

import csv

import traceback

import random

import re

from time import sleep

import os

from datetime import datetime, timedelta
import sys

In [2]:
requests.packages.urllib3.disable_warnings()

In [3]:
def get_original_weibo(info):
    """获取原创微博，青羊"""
    try:
        #rfind()返回字符串最后出现的位置，是一个数字，u是Unicode字符串
        #测试爬到了东西print(info.xpath('string(.)'))
        weibo_content = info.xpath('div//p')[0].xpath('string(.)').replace(u'\u200b','').encode(
            sys.stdout.encoding, 'ignore').decode(sys.stdout.encoding)
        if u'展开' in weibo_content:
            weibo_content = info.xpath('div//p')[1].xpath('string(.)').replace(u'\u200b','').replace('收起d','').encode(
                sys.stdout.encoding, 'ignore').decode(sys.stdout.encoding)
        return re.sub('\s|\t|\n','',weibo_content)
    except Exception as e:
        print('Error: ', e)
        traceback.print_exc()

In [7]:
#测试
'''
测试 mid 的 xpath 路径
测试 xpath('string(.)')的返回是什么 #是本页所有的text
测试是否获取到了原创微博（包括长微博）
'''
neirong = run(keyword,Cookie,User_Agent,start_time,end_time,pageNum)
print(neirong[3].xpath('@mid'))
print(get_original_weibo(neirong[3]))



第1页....



第2页....

['4887557072487652']

        
        
                        
                
                    
                
            
            
            
                
                    
                        c
                        
                            帮上头条
                            投诉
                            收藏
                                                                                                

                                                                                        复制微博地址
                        
                    
                    
                                                kylin麒麟法师
                                                

                        
                                            
                
                
                                                                                        
                        04月06日 15:57
                        
            

In [4]:
def get_publisher_info(user_id):
    """获取用户基本信息，青羊"""
    headers = {
        'Cookie': Cookie,
        'User-Agent': User_Agent,
        'Referer': 'https://weibo.com/{}?refer_flag=1001030103_'.format(user_id)
        }

    url = 'https://weibo.com/ajax/profile/info?custom={}'.format(user_id)
    response = requests.get(url=url, headers=headers)   
    #测试print(response.json())#爬取到了结果
    
    username = response.json()['data']['user']['screen_name']#昵称
    sex = response.json()['data']['user']['gender']#性别
    province = response.json()['data']['user']['location']#ip所在地
    following = response.json()['data']['user']['followers_count']#粉丝
    followed = response.json()['data']['user']['friends_count']#关注
    #测试print(username,sex,province,following,followed)#拿到了info

    return username, sex, province, following, followed

In [102]:
#测试
'''
测试是否拿到了info
昵称 性别 IP地址 粉丝数 关注数
'''
link = '//weibo.com/2260744952?refer_flag=1001030103_'
user_id = 2260744952
get_publisher_info(user_id)

步良猴Show-Beyond-吱 m 上海 241 1439


In [76]:
#测试
'''
测试能否通过re获取用户 user_id
'''
mstr = '//weibo.com/2260744952?refer_flag=1001030103_'
print(re.findall(re.compile(r'\d+'),mstr)[0])

2260744952


In [5]:
def is_original(mblog_id,user_id,keyword,Cookie,User_Agent):
    """获取微博信息，以及转发微博内容，青羊"""
    
    is_original = True
    content = ''
    text_raw = ''
    
    url = 'https://weibo.com/ajax/statuses/show?id={}'.format(mblog_id)   
    referer = 'https://weibo.com/{}/{}'.format(user_id,mblog_id)    
    headers = {
        'Cookie': Cookie,
        'User-Agent': User_Agent,
        'Referer': referer,
        'Connection':'close'
    }
    params = {
        'id': mblog_id
    }    
    res = requests.get(url=url, params=params, headers=headers,verify=False)
    #测试print(res.json())#爬取到了结果
    
    if 'repost_type' in res.json():
        is_original = False
        text_raw = res.json()['text_raw']
        content = text_raw[:text_raw.find('//@')]
    else:
        is_original = True
    if 'region_name' in res.json():
        place= res.json()['region_name'].replace('发布于','').replace('/s','')  # 微博发布位置
    else:
        place=''
    publish_time = res.json()['created_at']  # 微博发布时间
    #weibo['publish_tool'] = self.get_publish_tool(info)  # 微博发布工具
    up_num = res.json()['attitudes_count']  # 微博点赞数
    retweet_num = res.json()['comments_count']  # 转发数
    comment_num = res.json()['reposts_count']  # 评论数
    
    #测试print(is_original,place,publish_time,up_num,retweet_num,comment_num,text_raw,content)#拿到了内容
    return is_original,place,publish_time,up_num,retweet_num,comment_num,text_raw,content
    

In [87]:
#测试
'''
拿原微博链接
通过 re 获取原微博链接中的 mblog_id
测试结果多了一个？
'''
neirong = run(keyword,Cookie,User_Agent,start_time,end_time,pageNum)


aa=neirong[0].xpath(".//div[@class='from']/a/@href")[0]#原微博地址
print(aa)

print(re.compile(r'//weibo.com/(?P<user_id>.*?)/(?P<weibo_tag>.*?)refer_flag',re.S).search(aa).group('weibo_tag'))

//weibo.com/6979737921/MAId0m1J3?refer_flag=1001030103_
MAId0m1J3?


In [114]:
#测试
'''
测试 is_original()获取微博信息是否有效
'''
#非原创微博
#mblog_id = 'MB1Brgyqn'
#user_id = '1776018973'

#原创微博
mblog_id = 'MBjzcdmue'
user_id = '7742163338'

is_original(mblog_id,user_id,keyword,Cookie,User_Agent)

True  北京 Mon Apr 10 11:07:31 +0800 2023 0 0 0  


In [13]:
def get_one_weibo(info):
    """获取一条微博的全部信息，青羊"""
    #info是爬取到的部分html源码
    
    weibo = OrderedDict(user_id='',mblog_id='',publisher_name='',publisher_sex='',publisher_province='',\
                       publisher_following=0,publisher_followed=0,content='',text_raw='',\
                       publish_place='',publish_time='',up_num=0,retweet_num=0,comment_num=0,is_original=True)

    try:
        weibo_link = info.xpath(".//div[@class='from']/a/@href")[0]#原微博地址
        #print(weibo_link)
        publisher_link = info.xpath(".//div[@class='avator']/a/@href")[0]#发布者主页链接
        #print(publisher_link)
    
        obj= re.compile(r'//weibo.com/(?P<user_id>.*?)/(?P<mblog_id>.*?)refer_flag',re.S)
        weibo['mblog_id']=obj.search(weibo_link).group('mblog_id').rstrip('?')#微博mblog_id
        weibo['user_id'] = obj.search(weibo_link).group('user_id')#发布者id
    
        #微博基本信息
        weibo['is_original'], weibo['publish_place'], weibo['publish_time'], weibo['up_num'], \
        weibo['retweet_num'], weibo['comment_num'], weibo['text_raw'], weibo['content'] =\
        is_original(weibo['mblog_id'],weibo['user_id'],keyword,Cookie,User_Agent)
    
        if weibo['is_original']:
            weibo['content'] = get_original_weibo(info)  # 微博内容
        
        #用户基本信息
        weibo['publisher_name'], weibo['publisher_sex'], weibo['publisher_province'], \
        weibo['publisher_following'], weibo['publisher_followed'] = get_publisher_info(weibo['user_id'])

        return weibo
    except Exception as e:
        print('Error: ', e)
        traceback.print_exc()

In [7]:
def write_csv(weibo,wrote_num,got_num):
    """将爬取的信息写入csv文件，青羊"""
    try:
        result_headers = [
            '微博id',
            'mblog_id',
            '发布者昵称',
            '发布者性别',
            '发布者地区',
            '发布者关注数',
            '发布者粉丝数',
            '微博正文',
            '转发原文',
            '发布位置',
            '发布时间',
            '点赞数',
            '转发数',
            '评论数',
            '是否原创',
        ]

        result_data = [w.values() for w in weibo][wrote_num:]
        
        #测试打印，最终写入csv的内容
        print(result_data[0])
        
        #如果没有该文件，会自己创建一个 
        with open('data/raw_data/topic/' + keyword + '.csv', 'a', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            if wrote_num == 0:
                writer.writerows([result_headers])
            writer.writerows(result_data)
        print('%d条微博写入csv文件完毕:' % got_num)
    except Exception as e:
        print('Error: ', e)
        traceback.print_exc()

In [1]:
def run(keyword,Cookie,User_Agent,start_time,end_time,pageNum):

    wrote_num = 0#已写入的微博数
    page1 = 0#随机等待用的一个参数
    random_pages = random.randint(1, 5)#返回一个随机整型数,低包高不包，随即等待用的参数
    #pageNum = 4 #设置爬几页-1
    
    weibo = []# 存储爬取到的所有微博信息，list of dict
    got_num = 0  # 爬取到的微博数
    
    #爬取每一页微博
    for page in range(1, pageNum):
        
        print('\n\n第{}页....\n'.format(page))
        
        base_referer = 'https://s.weibo.com/realtime?'
        referer_params = {
            'q': keyword,
            'rd':start_time,
            'tw':end_time,
            'Refer':'weibo_realtime'
        }
        Referer = base_referer+urlencode(referer_params)

        headers = {
            'Cookie': Cookie,
            'User-Agent': User_Agent,
            'Referer': Referer,
            'Connection':'close'
        }
        params = {
            'q': keyword,
            'rd':start_time,
            'tw':end_time,
            'Refer':'weibo_realtime',
            'page': page
        }
        res = requests.get(url='https://s.weibo.com/realtime?', params=params, headers=headers,verify=False)
        #测试print(res.text)#爬到了东西
        
        html = etree.HTML(res.text)
        try:
            weibos = html.xpath("//div[@class='card-wrap'and @action-type='feed_list_item']")#内容来自html文件分层
            #测试print(len(weibos))#每一页10条左右

            # 自动结束
            if len(weibos) == 0:
                print('自动结束，大概率是因为内容爬完了，也请留意是否是 cookie 失效等情况\n')
                break

            #爬取本页每条微博
            for i in range(0, len(weibos)):
                aweibo = get_one_weibo(info=weibos[i])#爬取一条微博的信息
                if aweibo:
                    weibo.append(aweibo)
                    got_num += 1
                    print('-' * 20)

            if page % 3 == 0 and got_num > wrote_num:  # 每爬3页写入一次文件
                write_csv(weibo,wrote_num,got_num)
                wrote_num = got_num

            # 通过加入随机等待避免被限制。爬虫速度过快容易被系统限制(一段时间后限
            # 制会自动解除)，加入随机等待模拟人的操作，可降低被系统限制的风险。默
            # 认是每爬取1到5页随机等待6到10秒，如果仍然被限，可适当增加sleep时间
            if page - page1 == random_pages and page < pageNum:
                sleep(random.randint(6, 10))
                page1 = page
                # random_pages = random.randint(1, 5)
                random_pages = random.randint(1, 3)

        except:
            print(traceback.format_exc())

    if got_num > wrote_num:
        write_csv(weibo,wrote_num,got_num)  # 将剩余不足3页的微博写入文件
        
    print('共爬取' + str(got_num) + '条微博')
    

    #return weibos#测试接口

In [76]:
#测试
'''
查看爬到的html源码
检查两个容易出问题的xpath地址
两个xpath获取到的数字应当是有一部分重合，重合部分为 user_id
'''
result=run(keyword,Cookie,User_Agent,start_time,end_time,pageNum)[3]
print(etree.tostring(result,encoding='utf-8').decode('utf-8'))
print(result.xpath(".//div[@class='from']/a/@href"))#原微博地址
print(result.xpath(".//div[@class='avator']/a/@href"))#发布者主页链接
print(get_original_weibo(result))



第1页....

共爬取0条微博
<div class="card-wrap" action-type="feed_list_item" mid="4887532703319459">
        <div class="card">
        <div class="card-feed">
                        <div class="avator">
                <a href="//weibo.com/7453300571?refer_flag=1001030103_" target="_blank" suda-data="key=tblog_search_weibo&amp;value=seqid:1681119543372027384161|type:61|t:0|pos:1-0|q:%E5%8A%A0%E7%8F%AD%E6%96%87%E5%8C%96%E4%B8%BA%E4%BD%95%E5%B1%A1%E7%A6%81%E4%B8%8D%E6%AD%A2|ext:cate:31,mpos:1,click:user_pic"><img src="https://tvax1.sinaimg.cn/crop.0.0.1080.1080.180/0088phXlly8gemivr5royj30u00u0go0.jpg?KID=imgbed,tva&amp;Expires=1681130343&amp;ssig=OUnXrwbBCZ"/><i class="hoverMask"/>
                    <span title="微博个人认证" class="woo-icon-wrap woo-avatar-icon"><span class="woo-icon-main" style="width: 0.875rem; height: 0.875rem;"><svg class="woo-icon-in"><use xlink:href="#woo_svg_vgold"><svg xmlns="http://www.w3.org/2000/svg" viewbox="0 0 100 100" id="woo_svg_vgold"><path fill="#FDFF5D" d="M

In [5]:
#测试
'''
对 url 编码函数的测试
可以简化url编写方式
'''

keyword = '加班文化为何屡禁不止'
base_url = 'https://s.weibo.com/realtime?'
params = {
    'q': '#'+keyword+'#',#这个#不一定有，微博经常维护，有时候没有#，注意查看
    'rd': 'realtime',
    'tw': 'realtime',
    'Refer': 'weibo_realtime',
    'page': 2
}

print(base_url+urlencode(params))

https://s.weibo.com/realtime?q=%23%E5%8A%A0%E7%8F%AD%E6%96%87%E5%8C%96%E4%B8%BA%E4%BD%95%E5%B1%A1%E7%A6%81%E4%B8%8D%E6%AD%A2%23&rd=realtime&tw=realtime&Refer=weibo_realtime&page=2


In [11]:
# filter = 0 爬取所有微博，filter = 1 爬取原创微博#没写该功能
# 时间是从 start_time 到 end_time 这样
# 程序是从 end_time 到 start_time 这样爬


Cookie ='你的cookie’
User_Agent = '你的user_agent'

keyword = '加班文化为何屡禁不止'
start_time, end_time = '2023-04-06-07', '2023-04-06-23'
if start_time >= end_time:
    raise Exception('start_time 是离现在更远的那个时间，必须小于 end_time')#抛出一个err

In [14]:
#最终运行
run(keyword,Cookie,User_Agent,start_time,end_time,pageNum=5)



第1页....

--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------


第2页....

--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------


第3页....

--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
odict_values(['5619771281', 'MAIdwdwTe', '下饭专用', 'm', '广东', 164612, 512, '忍不住多说一句，先把加班费落实、节假日加班费翻倍、规定企业不能把加班费换成调休余额再来说为何屡禁不止吧！#加班文化为啥屡禁不止#', '', ' 广西', 'Thu Apr 06 12:02:29 +0800 2023', 69, 8, 1, True])
29条微博写入csv文件完毕:


第4页....

--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--

In [7]:
def time_params_formatter(params_time, offset_day=0, offset_hour=-8):
    '''一个时间加减函数，params_time是基准时间'''
    [temp_year, temp_month, temp_day, temp_hour] = [int(e) for e in params_time.split('-')]
    #测试print(temp_year)#测试是否分割成功
    temp_date = datetime(year=temp_year, month=temp_month, day=temp_day, hour=temp_hour)
    temp_offset = timedelta(days=offset_day, hours=offset_hour)
    res_time = (temp_date + temp_offset).strftime('%Y-%m-%d-%H')
    return res_time

In [9]:
#测试
'''
datetime的输出
time_params_formatter的作用
'''
print(datetime(2023,12,10,10))
print(time_params_formatter('2023-12-10-10',1,2))

2023-12-10 10:00:00
2023
2023-12-11-12
